## Общая структура объектов в Python

Прежде чем рассматривать конкретные числовые типы, важно понять, что в Python все объекты имеют общую структуру, которая включает:

- **Заголовок объекта**: Содержит информацию, общую для всех объектов:
  - **Счетчик ссылок** (`ob_refcnt`): Число ссылок на объект, используемое для управления памятью через подсчет ссылок.
  - **Указатель на тип** (`ob_type`): Указывает на объект типа (`PyTypeObject`), который описывает методы и операции, доступные для данного объекта.

- **Данные объекта**: Содержит специфичные для данного типа данные.

## Целые числа (`int`)

### Внутренняя реализация

В Python 3 целые числа представлены объектами типа `PyLongObject`, которые позволяют работать с числами **произвольной точности**. Это означает, что размер целого числа ограничен только доступной памятью.

**Структура `PyLongObject`:**

```c
typedef struct {
    PyObject_HEAD
    ssize_t ob_size;    // Число "цифр" в числе, знак определяется по об_size
    digit ob_digit[1];  // Массив "цифр" в выбранном основании
} PyLongObject;
```

- **`PyObject_HEAD`**: Заголовок объекта, содержащий `ob_refcnt` и `ob_type`.
- **`ob_size`**: Размер числа и знак:
  - Если `ob_size` > 0, число положительное.
  - Если `ob_size` < 0, число отрицательное.
  - Абсолютное значение `ob_size` — количество элементов в массиве `ob_digit`.
- **`ob_digit`**: Массив "цифр", где каждая "цифра" хранит часть числа в основании `2³⁰` (на 32-разрядных системах) или `2¹⁵` (на 16-разрядных системах).

### Пример

Рассмотрим число `12345678901234567890`.

- **Представление в памяти**:
  - Число разбивается на части по `30` бит (для 32-разрядной системы).
  - Массив `ob_digit` будет содержать несколько элементов, каждый из которых является частью числа.
- **Память**:
  - Заголовок объекта (`PyObject_HEAD`): содержит счетчик ссылок и указатель на тип.
  - Размер (`ob_size`): указывает на количество элементов в `ob_digit` и знак числа.
  - Данные (`ob_digit`): массив целых чисел типа `digit`, представляющих число.

## Вещественные числа с плавающей запятой (`float`)

### Внутренняя реализация

Тип `float` в Python соответствует типу `double` языка C и реализует стандарт IEEE 754 для чисел двойной точности (64 бита).

**Структура `PyFloatObject`:**

```c
typedef struct {
    PyObject_HEAD
    double ob_fval;  // Значение числа типа double
} PyFloatObject;
```

- **`PyObject_HEAD`**: Заголовок объекта.
- **`ob_fval`**: Число типа `double`, содержащее вещественное значение.

### Представление числа в памяти

- **64-битное число**: Представлено битами для знака, экспоненты и мантиссы:
  - **1 бит**: Знак числа.
  - **11 бит**: Экспонента.
  - **52 бита**: Мантисса.
  
Общая формула для восстановления числа:


$$\text{Value} = (-1)^S \times M \times 2^{E - Bias}$$

### Пример

- Число `3.1415` хранится в `ob_fval` в формате IEEE 754.
- Все операции с вещественными числами выполняются с использованием аппаратной поддержки процессора для типа `double`.

## Проблемы точности и округления

### Ограниченная точность представления

- Из-за конечного числа битов для мантиссы не все десятичные дроби могут быть точно представлены.
- Например, десятичное число 0.1 не может быть точно представлено в двоичной системе.


In [1]:
print(0.1 + 0.2)

0.30000000000000004


### Ошибки накопления

- При выполнении последовательности операций ошибки округления могут накапливаться, приводя к значительным отклонениям от ожидаемого результата.

### Проблемы сравнения

- Из-за ограниченной точности прямое сравнение вещественных чисел на равенство может быть ненадежным.
  
**Рекомендация:**

- Вместо проверки на точное равенство использовать проверку на **близость значений** с заданной точностью.

In [2]:
import math

def is_close(a, b, rel_tol=1e-9):
    return math.isclose(a, b, rel_tol=rel_tol)

In [3]:
0.1 + 0.2 == 0.3

False

In [4]:
is_close(0.1 + 0.2, 0.3)

True

## Пример представления числа в формате IEEE 754

Рассмотрим число `-5.75` в формате одинарной точности (32 бита).

1. **Знак (S)**: Поскольку число отрицательное, S = 1.

2. **Представление числа:**

   - Десятичное число `5.75` в двоичном виде: `101.11`
   - Нормализованная форма: `1.0111 × 2^2`

3. **Мантисса (M):**

   - Отбрасываем старший бит (скрытый бит 1), оставляем `01110000000000000000000` (23 бита)

4. **Порядок (E):**

   - Порядок `E` = экспонента + смещение
   - Экспонента = 2
   - Смещение (Bias) = 127
   - Тогда E = 2 + 127 = 129, в двоичном виде `10000001`

5. **Собираем все вместе:**

   - **Знак**: `1`
   - **Порядок**: `10000001`
   - **Мантисса**: `01110000000000000000000`

6. **Итоговое двоичное представление:**

   ```
   1 10000001 01110000000000000000000
   ```